In [4]:
import os
import sys
# //*** Imports and Load Data
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

install plaidml: https://towardsdatascience.com/plaidml-deep-learning-on-a-budget-ce22b58ba8c2

LTSM Code: https://medium.com/analytics-vidhya/beginners-guide-to-stock-prediction-using-lstm-7010bf8b8c21

In [6]:
#//*** Setup plaid keras
import plaidml.keras

import os

plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"


import keras

In [7]:
import keras
import keras.backend as K

K

<module 'keras.backend' from 'C:\\Users\\family\\AppData\\Roaming\\Python\\Python38\\site-packages\\keras\\backend.py'>

In [8]:
# 


#importing the libraries
import math
#import matplotlib.pyplot as plt
#import seaborn
import pandas as pd
import numpy as np
import datetime
import time
import requests
import io
from sklearn.preprocessing import MinMaxScaler
#from statsmodels.tsa.arima_model import ARIMA
#import statsmodels.api as sm
#import pandas_datareader as web
from sklearn.metrics import mean_squared_error
#Importing the keras model
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD

from keras.layers import LSTM
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor

In [9]:
#data_df = web.DataReader('APOLLOHOSP.NS', data_source = 'yahoo', start="2016-05-01", end="2021-05-01")
data_df = pd.read_csv("amc_daily.csv.zip")
data_df.head(10)

,date,symbol,open,high,low,close,volume
0,2021-06-30,amc,56.00,58.180,54.65,56.68,59020626
1,2021-06-29,amc,59.06,61.000,56.18,56.43,63604135
2,2021-06-28,amc,55.10,59.360,54.33,58.11,99310240
3,2021-06-25,amc,55.75,56.290,52.97,54.06,77596927
4,2021-06-24,amc,57.98,58.760,55.66,56.70,80351219
5,2021-06-23,amc,57.04,61.100,56.80,58.30,116291804
6,2021-06-22,amc,54.10,58.740,51.05,58.27,169494144
7,2021-06-21,amc,61.34,63.010,53.43,55.69,185876142
8,2021-06-18,amc,61.26,64.955,56.85,59.26,243645937
9,2021-06-17,amc,54.00,63.830,52.98,60.73,303576010


In [10]:
#Create a new dataframe with only the 'Adj close column'
new_data = data_df.filter(['close'])
#Convert the dataframe to a numpy array
dataset = new_data.to_numpy()
#Get the number of rows to train the model on
len_train = math.ceil(len(dataset)*.8)

train = dataset[0:len_train, :]
test = dataset[len_train:len(dataset), :1]

In [11]:
##Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1)) 
train_scaled = sc.fit_transform(train.reshape(-1,1))
test_scaled = sc.fit_transform(test.reshape(-1,1))

In [12]:
X_train, y_train = [], []
for i in range(len(train)-80-1):
    a = dataset[i:(i+80), 0]
    X_train.append(a)
    y_train.append(dataset[i+80, 0])
X_train = np.array(X_train)
y_train = np.array(y_train)

In [13]:
#Reshaping
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
#X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test = test.reshape(test.shape[0], test.shape[1], 1)

In [14]:
#create and fit the LSTM network
def model_lstm(lrn_rate = 0.01, mntum = 0):
    #Intializing the RNN
    model = Sequential()
    #1st layer
    model.add(LSTM(50, return_sequences=True, input_shape = (X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    #2nd layer
    model.add(LSTM(50))
    model.add(Dropout(0.2))
    #Adding the output layer
    model.add(Dense(units = 1))
    model.compile(loss = 'mean_squared_error', optimizer='adam')
    return model

In [16]:
LSTModel = model_lstm(lrn_rate=0.01, mntum=0)
LSTMmodel_fit = LSTModel.fit(X_train, y_train, batch_size=1, epochs=10)

Epoch 1/10
1436/1436 [==============================] - 55s 23ms/step - loss: 99.1333
Epoch 2/10
1436/1436 [==============================] - 32s 22ms/step - loss: 6.1959
Epoch 3/10
1436/1436 [==============================] - 32s 22ms/step - loss: 4.9676
Epoch 4/10
1436/1436 [==============================] - 32s 22ms/step - loss: 4.6528
Epoch 5/10
1436/1436 [==============================] - 32s 23ms/step - loss: 4.1321
Epoch 6/10
1436/1436 [==============================] - 33s 23ms/step - loss: 3.6993
Epoch 7/10
1436/1436 [==============================] - 32s 23ms/step - loss: 4.0608
Epoch 8/10
1436/1436 [==============================] - 32s 22ms/step - loss: 3.5643
Epoch 9/10
1436/1436 [==============================] - 32s 22ms/step - loss: 3.5221
Epoch 10/10
1436/1436 [==============================] - 33s 23ms/step - loss: 3.6189


In [20]:
closing_price = LSTModel.predict(X_test)
closing_price = sc.inverse_transform(closing_price)
closing_price

array([[268.1155 ],
       [268.0926 ],
       [268.05872],
       [267.90057],
       [267.9565 ],
       [267.9984 ],
       [267.9565 ],
       [267.96756],
       [267.9272 ],
       [267.961  ],
       [267.94742],
       [267.9945 ],
       [267.92963],
       [267.95312],
       [267.8818 ],
       [267.89923],
       [268.0023 ],
       [268.0328 ],
       [268.0337 ],
       [268.0543 ],
       [268.0602 ],
       [268.0427 ],
       [268.0497 ],
       [268.04504],
       [268.08054],
       [268.02075],
       [268.0497 ],
       [268.04425],
       [268.0117 ],
       [268.0849 ],
       [268.06448],
       [268.07602],
       [268.0558 ],
       [268.0849 ],
       [268.1128 ],
       [268.1239 ],
       [268.09027],
       [268.15723],
       [268.15704],
       [268.1549 ],
       [268.15613],
       [268.1554 ],
       [268.15594],
       [268.15088],
       [268.14526],
       [268.15652],
       [268.15097],
       [268.14053],
       [268.14526],
       [268.14975],


In [19]:
rms=np.sqrt(np.mean(np.power((y1-closing_price),2)))
rms

NameError: name 'y1' is not defined